In [ ]:
from google.colab import files
import pandas as pd
import io
import re

# 1. ЗАГРУЗКА ФАЙЛА
print("=== ЗАГРРУЗКА ФАЙЛА ===")
uploaded = files.upload()
print("Файл успешно загружен!")

# 2. ПОЛУЧАЕМ ТОЧНОЕ ИМЯ ФАЙЛА
print("\n=== ПРОВЕРКА ЗАГРУЖЕННЫХ ФАЙЛОВ ===")
print("Загруженные файлы:")
for filename in uploaded.keys():
    print(f"  - '{filename}'")

# Берем первое (и единственное) имя файла
filename = list(uploaded.keys())[0]
print(f"\nБудем использовать файл: '{filename}'")

# 3. ЧТЕНИЕ ФАЙЛА С ПРАВИЛЬНЫМ РАЗДЕЛИТЕЛЕМ
print("\n=== ЧТЕНИЕ ДАННЫХ ===")
try:
    # Используем точное имя файла
    df = pd.read_csv(io.BytesIO(uploaded[filename]), sep='\t', encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv(io.BytesIO(uploaded[filename]), sep='\t', encoding='latin-1')

print(f"Размер исходной таблицы: {df.shape}")
print("Сырые названия столбцов:")
for i, col in enumerate(df.columns):
    print(f"  {i}: '{col}'")

# 4. ОЧИСТКА НАЗВАНИЙ СТОЛБЦОВ
print("\n=== ОЧИСТКА НАЗВАНИЙ СТОЛБЦОВ ===")

def clean_column_name(col):
    """Функция для очистки названий столбцов от специальных символов"""
    # Убираем табуляции, переносы строк, кавычки для того, чтобы они не сбивали анализ
    col = re.sub(r'[\t\n\r"]', '', col)
    # Убираем лишние пробелы
    col = re.sub(r'\s+', ' ', col)
    # Убираем пробелы по краям
    col = col.strip()
    return col

# Применяем очистку ко всем столбцам
df.columns = [clean_column_name(col) for col in df.columns]

print("Очищенные названия столбцов:")
for i, col in enumerate(df.columns):
    print(f"  {i}: '{col}'")

# 5. ОЧИСТКА ДАННЫХ В СТОЛБЦАХ
print("\n=== ОЧИСТКА ДАННЫХ ===")

# Функция для очистки значений в столбцах
def clean_data(value):
    if isinstance(value, str):
        # Убираем табуляции и лишние пробелы
        value = re.sub(r'[\t\n\r]', '', value)
        value = re.sub(r'\s+', ' ', value)
        value = value.strip()
        # Пробуем преобразовать к числу если возможно
        try:
            # Проверяем, является ли строка числом
            if value.replace('.', '').replace('-', '').isdigit():
                return float(value) if '.' in value else int(value)
            else:
                return value
        except ValueError:
            return value
    return value

# Применяем очистку ко всем столбцам
for col in df.columns:
    df[col] = df[col].apply(clean_data)

# 6. ПРЕОБРАЗОВАНИЕ БУКВЕННЫХ ЗНАЧЕНИЙ В ЧИСЛА, потому что компьютер букв не понимает
print("\n=== ПРЕОБРАЗОВАНИЕ КАТЕГОРИАЛЬНЫХ ПЕРЕМЕННЫХ ===")

# Словари для преобразования
gender_mapping = {
    'male': 0,
    'female': 1,
    'non-binary/other': 2,
    'Male': 0,
    'Female': 1,
    'Non-binary/Other': 2
}

occupation_mapping = {
    'employed': 0,
    'student': 1,
    'self-employed': 2,
    'retired': 3,
    'unemployed': 4,
    'Employed': 0,
    'Student': 1,
    'Self-employed': 2,
    'Retired': 3,
    'Unemployed': 4
}

work_mode_mapping = {
    'remote': 0,
    'in-person': 1,
    'hybrid': 2,
    'Remote': 0,
    'In-person': 1,
    'Hybrid': 2
}

# Проверяем какие столбцы есть в данных и преобразуем их
print("Преобразование категориальных переменных:")

if 'gender' in df.columns:
    print("  - Преобразование gender...")
    df['gender'] = df['gender'].str.lower().map(gender_mapping)
    # Заполняем пропущенные значения (если есть) значением по умолчанию
    df['gender'] = df['gender'].fillna(2)  # Non-binary/Other по умолчанию
    print(f"    Уникальные значения после преобразования: {df['gender'].unique()}")

if 'occupation' in df.columns:
    print("  - Преобразование occupation...")
    df['occupation'] = df['occupation'].str.lower().map(occupation_mapping)
    df['occupation'] = df['occupation'].fillna(0)  # Employed по умолчанию
    print(f"    Уникальные значения после преобразования: {df['occupation'].unique()}")

if 'work_mode' in df.columns:
    print("  - Преобразование work_mode...")
    df['work_mode'] = df['work_mode'].str.lower().map(work_mode_mapping)
    df['work_mode'] = df['work_mode'].fillna(1)  # In-person по умолчанию
    print(f"    Уникальные значения после преобразования: {df['work_mode'].unique()}")

# 7. УДАЛЕНИЕ ПУСТЫХ СТОЛБЦОВ, чтобы не замедляли и не затрудняли анализ
print("\n=== УДАЛЕНИЕ ПУСТЫХ СТОЛБЦОВ ===")
initial_columns = len(df.columns)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
final_columns = len(df.columns)
print(f"Удалено пустых столбцов: {initial_columns - final_columns}")

# 8. ФИНАЛЬНАЯ ПРОВЕРКА
print("\n=== РЕЗУЛЬТАТ ===")
print(f"Финальный размер таблицы: {df.shape}")
print(f"Количество строк: {df.shape[0]}")
print(f"Количество столбцов: {df.shape[1]}")
print("\nНазвания столбцов:")
print(df.columns.tolist())

print("\nТипы данных:")
print(df.dtypes)

print("\nПервые 5 строк данных:")
pd.set_option('display.max_columns', None)  # Показывать все столбцы
print(df.head())

print("\nПроверка пропущенных значений:")
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])  # Показываем только столбцы с пропусками

# Проверяем преобразованные столбцы
print("\n=== ПРОВЕРКА ПРЕОБРАЗОВАННЫХ СТОЛБЦОВ ===")
if 'gender' in df.columns:
    print(f"Gender - уникальные значения: {sorted(df['gender'].unique())}")
if 'occupation' in df.columns:
    print(f"Occupation - уникальные значения: {sorted(df['occupation'].unique())}")
if 'work_mode' in df.columns:
    print(f"Work_mode - уникальные значения: {sorted(df['work_mode'].unique())}")

print("\nОсновная статистика для числовых столбцов:")
print(df.describe())

# 9. СОХРАНЕНИЕ ОЧИЩЕННОЙ ТАБЛИЦЫ
print("\n=== СОХРАНЕНИЕ ===")
# Сохраняем очищенную таблицу в Colab
clean_filename = f"cleaned_{filename}"
df.to_csv(clean_filename, index=False)
print(f"Очищенная таблица сохранена как '{clean_filename}'")

# Сохраняем также версию с описанием преобразований
with open('data_mapping_info.txt', 'w') as f:
    f.write("Преобразования категориальных переменных:\n")
    f.write("Gender: male=0, female=1, non-binary/other=2\n")
    f.write("Occupation: Employed=0, Student=1, Self-employed=2, Retired=3, Unemployed=4\n")
    f.write("Work mode: Remote=0, In-person=1, Hybrid=2\n")
print("Информация о преобразованиях сохранена в 'data_mapping_info.txt'")

# Скачиваем на компьютер (опционально)
files.download(clean_filename)
files.download('data_mapping_info.txt')
print("Файлы готовы к скачиванию!")

=== ЗАГРРУЗКА ФАЙЛА ===


Saving DB_3.csv to DB_3 (4).csv
Файл успешно загружен!

=== ПРОВЕРКА ЗАГРУЖЕННЫХ ФАЙЛОВ ===
Загруженные файлы:
  - 'DB_3 (4).csv'

Будем использовать файл: 'DB_3 (4).csv'

=== ЧТЕНИЕ ДАННЫХ ===
Размер исходной таблицы: (400, 16)
Сырые названия столбцов:
  0: 'user_id	'
  1: 'age'
  2: 'gender'
  3: 'occupation'
  4: 'work_mode'
  5: 'screen_time_hours	'
  6: 'work_screen_hours'
  7: 'leisure_screen_hours'
  8: 'sleep_hours'
  9: 'sleep_quality_1_5'
  10: 'stress_level_0_10'
  11: 'productivity_0_100	'
  12: 'exercise_minutes_per_week'
  13: 'social_hours_per_week'
  14: 'mental_wellness_index_0_100'
  15: 'Unnamed: 15'

=== ОЧИСТКА НАЗВАНИЙ СТОЛБЦОВ ===
Очищенные названия столбцов:
  0: 'user_id'
  1: 'age'
  2: 'gender'
  3: 'occupation'
  4: 'work_mode'
  5: 'screen_time_hours'
  6: 'work_screen_hours'
  7: 'leisure_screen_hours'
  8: 'sleep_hours'
  9: 'sleep_quality_1_5'
  10: 'stress_level_0_10'
  11: 'productivity_0_100'
  12: 'exercise_minutes_per_week'
  13: 'social_hours_per_w

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Файлы готовы к скачиванию!


In [1]:
# ЯЧЕЙКА 1: Импорт библиотек и загрузка файла
from google.colab import files
import pandas as pd
import io
import re
import numpy as np

print("=== ЗАГРУЗКА ФАЙЛА ===")
uploaded = files.upload()

# Получаем точное имя файла
filename = list(uploaded.keys())[0]
print(f"Загружен файл: '{filename}'")

=== ЗАГРУЗКА ФАЙЛА ===


Saving DB_3.csv to DB_3.csv
Загружен файл: 'DB_3.csv'


In [2]:
# ЯЧЕЙКА 2: Чтение данных и очистка названий столбцов

# Чтение файла с правильным разделителем
try:
    df = pd.read_csv(io.BytesIO(uploaded[filename]), sep='\t', encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv(io.BytesIO(uploaded[filename]), sep='\t', encoding='latin-1')

print(f"Исходный размер таблицы: {df.shape}")

# Очистка названий столбцов, чтобы не затруднялся анализ
# Убираем табуляции, переносы строк, кавычки для того, чтобы они не сбивали анализ
def clean_column_name(col):
    col = re.sub(r'[\t\n\r"]', '', col)
    col = re.sub(r'\s+', ' ', col)
    return col.strip()

df.columns = [clean_column_name(col) for col in df.columns]

print("Очищенные названия столбцов:")
for i, col in enumerate(df.columns):
    print(f"  {i}: '{col}'")

Исходный размер таблицы: (400, 16)
Очищенные названия столбцов:
  0: 'user_id'
  1: 'age'
  2: 'gender'
  3: 'occupation'
  4: 'work_mode'
  5: 'screen_time_hours'
  6: 'work_screen_hours'
  7: 'leisure_screen_hours'
  8: 'sleep_hours'
  9: 'sleep_quality_1_5'
  10: 'stress_level_0_10'
  11: 'productivity_0_100'
  12: 'exercise_minutes_per_week'
  13: 'social_hours_per_week'
  14: 'mental_wellness_index_0_100'
  15: 'Unnamed: 15'


In [3]:
# ЯЧЕЙКА 3: Очистка данных в ячейках

def clean_data(value):
    if isinstance(value, str):
        value = re.sub(r'[\t\n\r]', '', value)
        value = re.sub(r'\s+', ' ', value)
        value = value.strip()
        try:
            if value.replace('.', '').replace('-', '').isdigit():
                return float(value) if '.' in value else int(value)
            else:
                return value
        except ValueError:
            return value
    return value

# Применяем очистку ко всем столбцам
for col in df.columns:
    df[col] = df[col].apply(clean_data)

print("Данные очищены!")
print(f"Типы данных после очистки:")
print(df.dtypes)

Данные очищены!
Типы данных после очистки:
user_id                         object
age                              int64
gender                          object
occupation                      object
work_mode                       object
screen_time_hours              float64
work_screen_hours              float64
leisure_screen_hours           float64
sleep_hours                    float64
sleep_quality_1_5              float64
stress_level_0_10              float64
productivity_0_100             float64
exercise_minutes_per_week        int64
social_hours_per_week          float64
mental_wellness_index_0_100    float64
Unnamed: 15                    float64
dtype: object


In [4]:
# ЯЧЕЙКА 4: ПРЕОБРАЗОВАНИЕ БУКВЕННЫХ ЗНАЧЕНИЙ В ЧИСЛА, потому что компьютер букв не понимает

# Словари для преобразования
gender_mapping = {
    'male': 0, 'female': 1, 'non-binary/other': 2,
    'Male': 0, 'Female': 1, 'Non-binary/Other': 2
}

occupation_mapping = {
    'employed': 0, 'student': 1, 'self-employed': 2, 'retired': 3, 'unemployed': 4,
    'Employed': 0, 'Student': 1, 'Self-employed': 2, 'Retired': 3, 'Unemployed': 4
}

work_mode_mapping = {
    'remote': 0, 'in-person': 1, 'hybrid': 2,
    'Remote': 0, 'In-person': 1, 'Hybrid': 2
}

print("=== ПРЕОБРАЗОВАНИЕ КАТЕГОРИАЛЬНЫХ ПЕРЕМЕННЫХ ===")

# Преобразование gender
if 'gender' in df.columns:
    df['gender'] = df['gender'].str.lower().map(gender_mapping).fillna(2)
    print(f"Gender преобразован: {df['gender'].unique()}")

# Преобразование occupation
if 'occupation' in df.columns:
    df['occupation'] = df['occupation'].str.lower().map(occupation_mapping).fillna(0)
    print(f"Occupation преобразован: {df['occupation'].unique()}")

# Преобразование work_mode
if 'work_mode' in df.columns:
    df['work_mode'] = df['work_mode'].str.lower().map(work_mode_mapping).fillna(1)
    print(f"Work_mode преобразован: {df['work_mode'].unique()}")

=== ПРЕОБРАЗОВАНИЕ КАТЕГОРИАЛЬНЫХ ПЕРЕМЕННЫХ ===
Gender преобразован: [1. 0. 2.]
Occupation преобразован: [0. 1. 2. 3. 4.]
Work_mode преобразован: [0. 1. 2.]


In [5]:
# ЯЧЕЙКА 5: Удаление пустых столбцов, чтобы они не затрудняли анализ и финальная проверка

# Удаление пустых столбцов
initial_columns = len(df.columns)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
final_columns = len(df.columns)
print(f"Удалено пустых столбцов: {initial_columns - final_columns}")

print("=== ФИНАЛЬНАЯ ПРОВЕРКА ===")
print(f"Размер таблицы: {df.shape}")
print("\nНазвания столбцов:")
print(df.columns.tolist())

print("\nПервые 3 строки данных:")
pd.set_option('display.max_columns', None)
print(df.head(3))

print("\nПропущенные значения:")
print(df.isnull().sum())

Удалено пустых столбцов: 1
=== ФИНАЛЬНАЯ ПРОВЕРКА ===
Размер таблицы: (400, 15)

Названия столбцов:
['user_id', 'age', 'gender', 'occupation', 'work_mode', 'screen_time_hours', 'work_screen_hours', 'leisure_screen_hours', 'sleep_hours', 'sleep_quality_1_5', 'stress_level_0_10', 'productivity_0_100', 'exercise_minutes_per_week', 'social_hours_per_week', 'mental_wellness_index_0_100']

Первые 3 строки данных:
  user_id  age  gender  occupation  work_mode  screen_time_hours  \
0   U0001   33     1.0         0.0        0.0              10.79   
1   U0002   28     1.0         0.0        1.0               7.40   
2   U0003   35     1.0         0.0        2.0               9.78   

   work_screen_hours  leisure_screen_hours  sleep_hours  sleep_quality_1_5  \
0               5.44                  5.35         6.63                1.0   
1               0.37                  7.03         8.05                3.0   
2               1.09                  8.69         6.48                1.0   

   

In [6]:
# ЯЧЕЙКА 5: Обработка пропущенных значений в user_id, чтобы эти данные могли пойти в анализ

print("=== ОБРАБОТКА USER_ID ===")

if 'user_id' in df.columns:
    # Находим пропущенные значения
    missing_user_ids = df['user_id'].isnull().sum()
    print(f"Найдено пропущенных user_id: {missing_user_ids}")

    if missing_user_ids > 0:
        # Находим существующие user_id и извлекаем номера
        existing_ids = df['user_id'].dropna()

        if len(existing_ids) > 0:
            # Извлекаем числовую часть из существующих ID
            def extract_id_number(user_id):
                if isinstance(user_id, str) and user_id.startswith('U'):
                    try:
                        # Ищем числовую часть после 'U'
                        import re
                        numbers = re.findall(r'\d+', user_id)
                        if numbers:
                            return int(numbers[0])
                    except:
                        return 0
                return 0

            # Находим максимальный номер ID
            id_numbers = existing_ids.apply(extract_id_number)
            max_id = id_numbers.max()
            print(f"Максимальный найденный номер ID: {max_id}")

            # Заполняем пропущенные значения
            missing_count = 0
            for idx in df[df['user_id'].isnull()].index:
                missing_count += 1
                new_id_number = max_id + missing_count
                new_user_id = f"U{new_id_number:04d}"  # Формат U0001, U0002 и т.д.
                df.at[idx, 'user_id'] = new_user_id
                print(f"Заполнен пропуск: {new_user_id}")

            print(f"Всего заполнено {missing_count} пропущенных user_id")
        else:
            # Если нет существующих ID, начинаем с U0001
            missing_count = 0
            for idx in df[df['user_id'].isnull()].index:
                missing_count += 1
                df.at[idx, 'user_id'] = f"U{missing_count:04d}"
            print(f"Создано {missing_count} новых user_id (с U0001)")
    else:
        print("Пропущенных user_id не найдено")

    # Проверяем результат
    print(f"\nУникальные user_id после обработки (первые 10):")
    print(df['user_id'].dropna().unique()[:10])
else:
    print("Столбец 'user_id' не найден в данных")

=== ОБРАБОТКА USER_ID ===
Найдено пропущенных user_id: 2
Максимальный найденный номер ID: 400
Заполнен пропуск: U0401
Заполнен пропуск: U0402
Всего заполнено 2 пропущенных user_id

Уникальные user_id после обработки (первые 10):
['U0001' 'U0002' 'U0003' 'U0004' 'U0005' 'U0006' 'U0007' 'U0008' 'U0009'
 'U0010']


In [7]:
# ЯЧЕЙКА 5: Заполнение пропущенных значений медианой

print("=== ЗАПОЛНЕНИЕ ПРОПУЩЕННЫХ ЗНАЧЕНИЙ МЕДИАНОЙ ===")

# Создаем копию DataFrame для безопасной работы
df_filled = df.copy()

# Счетчики для статистики
total_missing_before = df_filled.isnull().sum().sum()
columns_with_missing = df_filled.columns[df_filled.isnull().any()].tolist()

print(f"Общее количество пропущенных значений до обработки: {total_missing_before}")
print(f"Столбцы с пропущенными значениями: {columns_with_missing}")

# Заполняем пропущенные значения медианой для числовых столбцов
for column in df_filled.columns:
    if df_filled[column].isnull().sum() > 0:
        missing_count = df_filled[column].isnull().sum()

        # Проверяем, является ли столбец числовым
        if pd.api.types.is_numeric_dtype(df_filled[column]):
            # Вычисляем медиану (игнорируя пропущенные значения)
            column_median = df_filled[column].median()

            # Заполняем пропуски медианой
            df_filled[column].fillna(column_median, inplace=True)

            print(f"✅ Столбец '{column}':")
            print(f"   - Заполнено {missing_count} пропусков")
            print(f"   - Медиана: {column_median:.2f}")
            print(f"   - Пропусков после: {df_filled[column].isnull().sum()}")
        else:
            # Для нечисловых столбцов используем моду (самое частое значение)
            if df_filled[column].notna().any():  # Если есть непустые значения
                column_mode = df_filled[column].mode()
                if len(column_mode) > 0:
                    most_frequent = column_mode[0]
                    df_filled[column].fillna(most_frequent, inplace=True)
                    print(f"📊 Столбец '{column}' (нечисловой):")
                    print(f"   - Заполнено {missing_count} пропусков значением: '{most_frequent}'")
                else:
                    print(f"⚠️  Столбец '{column}': невозможно определить моду")
            else:
                print(f"⚠️  Столбец '{column}': все значения пропущены")

# Обновляем основной DataFrame
df = df_filled.copy()

# Финальная статистика
total_missing_after = df.isnull().sum().sum()
print(f"\n=== РЕЗУЛЬТАТ ===")
print(f"Пропущенные значения ДО обработки: {total_missing_before}")
print(f"Пропущенные значения ПОСЛЕ обработки: {total_missing_after}")
print(f"Удалено пропусков: {total_missing_before - total_missing_after}")

# Показываем столбцы, где еще остались пропуски (если есть)
remaining_missing = df.columns[df.isnull().any()].tolist()
if remaining_missing:
    print(f"\n⚠️  Столбцы с оставшимися пропусками: {remaining_missing}")
    for col in remaining_missing:
        print(f"   - {col}: {df[col].isnull().sum()} пропусков")
else:
    print("🎉 Все пропущенные значения успешно заполнены!")

=== ЗАПОЛНЕНИЕ ПРОПУЩЕННЫХ ЗНАЧЕНИЙ МЕДИАНОЙ ===
Общее количество пропущенных значений до обработки: 11
Столбцы с пропущенными значениями: ['screen_time_hours', 'work_screen_hours', 'leisure_screen_hours', 'sleep_quality_1_5', 'stress_level_0_10', 'mental_wellness_index_0_100']
✅ Столбец 'screen_time_hours':
   - Заполнено 2 пропусков
   - Медиана: 9.09
   - Пропусков после: 0
✅ Столбец 'work_screen_hours':
   - Заполнено 3 пропусков
   - Медиана: 1.46
   - Пропусков после: 0
✅ Столбец 'leisure_screen_hours':
   - Заполнено 1 пропусков
   - Медиана: 6.70
   - Пропусков после: 0
✅ Столбец 'sleep_quality_1_5':
   - Заполнено 2 пропусков
   - Медиана: 1.00
   - Пропусков после: 0
✅ Столбец 'stress_level_0_10':
   - Заполнено 1 пропусков
   - Медиана: 8.80
   - Пропусков после: 0
✅ Столбец 'mental_wellness_index_0_100':
   - Заполнено 2 пропусков
   - Медиана: 14.80
   - Пропусков после: 0

=== РЕЗУЛЬТАТ ===
Пропущенные значения ДО обработки: 11
Пропущенные значения ПОСЛЕ обработки: 0
Удал

/tmp/ipython-input-2496174412.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_filled[column].fillna(column_median, inplace=True)


In [8]:
# ЯЧЕЙКА 7: Сохранение и скачивание результата

print("=== СОХРАНЕНИЕ РЕЗУЛЬТАТОВ ===")

# Сохраняем очищенный файл
cleaned_filename = f"cleaned_{filename}"
df.to_csv(cleaned_filename, index=False)
print(f"✅ Очищенная таблица сохранена как '{cleaned_filename}'")

# Скачиваем на компьютер
files.download(cleaned_filename)
print("📥 Файл скачан на ваш компьютер!")

# Дополнительно: сохраняем информацию о преобразованиях
info_content = f"""Информация о преобразованиях данных:
Дата обработки: {pd.Timestamp.now()}
Исходный файл: {filename}
Размер данных: {df.shape[0]} строк, {df.shape[1]} столбцов

Преобразования категориальных переменных:
- gender: male=0, female=1, non-binary/other=2
- occupation: Employed=0, Student=1, Self-employed=2, Retired=3, Unemployed=4
- work_mode: Remote=0, In-person=1, Hybrid=2
"""

with open('data_transformation_info.txt', 'w') as f:
    f.write(info_content)

files.download('data_transformation_info.txt')
print("📄 Информация о преобразованиях скачана!")

=== СОХРАНЕНИЕ РЕЗУЛЬТАТОВ ===
✅ Очищенная таблица сохранена как 'cleaned_DB_3.csv'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 Файл скачан на ваш компьютер!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📄 Информация о преобразованиях скачана!
